In [1]:
import numpy
from etrade.spot.trader import Station
from etrade.spot.market.recycle import BasicRecycle, PointwiseRecycle
from etrade.spot.forecast.market import DistributiveSeries, DistributiveMarket
from  data_utils.stochastic_utils.vdistributions.abstract import AbstractDistribution
from  data_utils.stochastic_utils.vdistributions.parameter.continuous.basic import NormalDistribution, SkewNormalDistribution
from data_utils.stochastic_utils.vdistributions.tools.divergence import kl_divergence_continuous, crps

from data_utils.stochastic_utils.vdistributions.nonparametric.continuous.mfk.skewnd2 import dof_snd_fitter

from scipy.optimize import differential_evolution

from matplotlib import pyplot

In [2]:
s = Station("s", 50)
br = PointwiseRecycle(0.8, 1.5)

In [3]:
real_market = DistributiveMarket(
    DistributiveSeries(NormalDistribution(45, 5), NormalDistribution(46, 4), NormalDistribution(30, 5), NormalDistribution(32, 5)),
    DistributiveSeries(NormalDistribution(10, 2), NormalDistribution(11, 3), NormalDistribution(5, 1), NormalDistribution(12, 5)),
    DistributiveSeries(NormalDistribution(10, 3), NormalDistribution(18, 9), NormalDistribution(5, 2), NormalDistribution(13, 6)),
)
per_market = DistributiveMarket(
    DistributiveSeries(NormalDistribution(45, 5), NormalDistribution(46, 4), NormalDistribution(30, 5), NormalDistribution(32, 5)),
    DistributiveSeries(NormalDistribution(10, 2), NormalDistribution(11, 3), NormalDistribution(5, 1), NormalDistribution(12, 5)),
    DistributiveSeries(NormalDistribution(10, 3), NormalDistribution(18, 9), NormalDistribution(5, 2), NormalDistribution(13, 6)),
)

In [4]:
train_aq, train_dp, train_rp = per_market.rvf(1000)
optimied = per_market.submitted_quantity_optimizer(s, br, train_aq, train_dp, train_rp)

In [5]:
print(optimied.x)


[34.40536788 32.38389314 23.63316809 25.88709099]


In [6]:
real = real_market.rvf(1)
s.trade(real[0].reshape(4), optimied.x, real[1].reshape(4), real[2].reshape(4))
# br(real[0].reshape(4), optimied.x, s.trade(real[0].reshape(4), optimied.x, real[1].reshape(4), real[2].reshape(4)))

array([516.08304277, 462.46415373, 116.36354406, 655.3873565 ])

In [7]:
numpy.sum(br(
    real[0], 
    optimied.x, 
    s.trade(real[0].reshape(4), optimied.x, real[1].reshape(4), real[2].reshape(4))
))

1750.2980970644546

In [8]:
numpy.sum(br(
    real[0], 
    real[0].reshape(4), 
    s.trade(real[0].reshape(4), real[0].reshape(4), real[1].reshape(4), real[2].reshape(4))
))

1818.513672206822

In [9]:
numpy.concatenate((per_market.crps(*real), [per_market.price_kl_divergence()])).reshape(-1)

array([7.37551271, 4.61552012, 1.17609478, 1.50338407, 0.4679783 ,
       0.75713712, 0.40079722, 3.54467118, 0.81072668, 8.66113234,
       1.75806248, 7.75536892, 0.12768733, 0.95663698, 0.31814718,
       0.04343267])

In [10]:
print(per_market.price_kl_divergence())

[0.12768733 0.95663698 0.31814718 0.04343267]


In [11]:
kl_divergence_continuous(NormalDistribution(0,1), NormalDistribution(0,10))

1.807585092998198

In [12]:
kl_divergence_continuous(NormalDistribution(0,1), SkewNormalDistribution(0,1,1))

0.30684806434836825